In [76]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from datetime import date,time,datetime


In [32]:
url = 'https://google.com'

response = requests.get(url)
print(response.status_code)
# print(response.content)

soup = BeautifulSoup(response.content,'html.parser')

# print(dir(soup))

# print(soup.prettify()) #display response in a pretty format

print(soup.title.text)

var = soup.find('div',{'id':'gbar'})

search = var.find_all('a',{'class':'gb1'})
# print(search)

list = []

for text in search:
    list.append(text.text)

print(list)

200
Google
['Images', 'Maps', 'Play', 'YouTube', 'News', 'Gmail', 'Drive', 'More »']


In [58]:
#LinkedIn job scraper
list=[]

url = 'https://linkedin.com/jobs/search'

params = {
    'distance':'10',
    'f_TPR':'r86400',
    'f_TW':'3',
    'geoId':'105307040',
    'location':'Bengaluru',
    'start':'0'
}

for page in range(0,1000,25):

    params.update({'start':str(page)})

    response = requests.get(url=url,params=params)
    # print(response.status_code)
    # print(response.url)

    soup = BeautifulSoup(response.content,'html.parser')

    # print(soup.prettify())

    jobs_list = soup.find('ul',{'class':'jobs-search__results-list'})

    jobs = jobs_list.find_all('li')

    # print(jobs)

    for job in jobs:
        try:
            dict = {}
            dict['Title'] = job.find('h3').text.strip()
            dict['Company'] = job.find('h4').text.strip()
            dict['Company Link']= job.find('a',{'data-tracking-client-ingraph':''})['href']
            dict['Location'] = job.find('span',{'class':'job-search-card__location'}).text.strip()
            dict['Status'] = job.find('span',{'class':'result-benefits__text'}).text.strip()
            dict['Posted'] = job.find('time').text.strip()
        except:
            pass
        list.append(dict)

print(len(list))


1000


In [116]:
print(len(list))#all pages
df = pd.DataFrame(list)
df

1000


,Title,Company,Company Link,Location,Status,Posted
0,Sales Specialist,Amazon,https://in.linkedin.com/jobs/view/sales-specia...,"Bengaluru, Karnataka, India",Actively Hiring,19 hours ago
1,Financial Analyst,Rolls-Royce,https://in.linkedin.com/jobs/view/financial-an...,"Bengaluru, Karnataka, India",Actively Hiring,40 minutes ago
2,store accelerator programme: business trainee,CRED,https://in.linkedin.com/jobs/view/store-accele...,"Bengaluru, Karnataka, India",Be an early applicant,9 hours ago
3,Sales Manager I,Swiggy,https://in.linkedin.com/jobs/view/sales-manage...,"Bengaluru, Karnataka, India",Actively Hiring,14 hours ago
4,Bioinformatics part time job/internship at Ban...,BayesLabs,https://in.linkedin.com/jobs/view/bioinformati...,"Bengaluru, Karnataka, India",Be an early applicant,17 hours ago
...,...,...,...,...,...,...
995,Analyst,PwC India,https://in.linkedin.com/jobs/view/analyst-at-p...,"Bengaluru, Karnataka, India",Actively Hiring,18 hours ago
996,Customer Support Intern,Truecaller,https://in.linkedin.com/jobs/view/customer-sup...,"Bengaluru, Karnataka, India",Be an early applicant,6 minutes ago
997,Strategic Sourcing and Vendor Management,Yulu,https://in.linkedin.com/jobs/view/strategic-so...,"Bengaluru, Karnataka, India",Actively Hiring,22 hours ago
998,"Software Engineer (Backend, 7-10 years)",PhonePe,https://in.linkedin.com/jobs/view/software-eng...,"Bengaluru, Karnataka, India",Actively Hiring,15 hours ago


In [117]:
df = df.drop_duplicates()
df.shape
df['Posted'].unique()

array(['19 hours ago', '40 minutes ago', '9 hours ago', '14 hours ago',
       '17 hours ago', '13 hours ago', '10 hours ago', '2 hours ago',
       '6 hours ago', '21 hours ago', '53 minutes ago', '1 hour ago',
       '22 hours ago', '12 hours ago', '5 minutes ago', '18 hours ago',
       '15 hours ago', '20 hours ago', '7 hours ago', '41 minutes ago',
       '45 minutes ago', '11 hours ago', '16 hours ago', '36 minutes ago',
       '4 hours ago', '15 minutes ago', '23 hours ago', '54 minutes ago',
       '56 minutes ago', '8 hours ago', nan, '35 minutes ago',
       '3 hours ago', '6 minutes ago', '42 minutes ago'], dtype=object)

In [118]:
# To convert posted info to actuall date and time of posted
import datetime

def convert_post(posted):

    now = datetime.datetime.now()
    post = re.compile('\d+')
    try:
        value = post.search(posted).group()
    except:
        value = '0'
    time = int(value)

    if 'hour' or 'hours' in posted:
        result = now - datetime.timedelta(hours=time)
    elif 'minute' or 'minutes' in posted:
        mins = time/60.0
        result = now - datetime.timedelta(hours=mins)
    elif 'days' or 'day' in posted:
        result = now - datetime.timedelta(days = time)
    else:
        print("Noting to convert")
        
    return result


# print(convert_post('3 days ago'))
df
    

,Title,Company,Company Link,Location,Status,Posted
0,Sales Specialist,Amazon,https://in.linkedin.com/jobs/view/sales-specia...,"Bengaluru, Karnataka, India",Actively Hiring,19 hours ago
1,Financial Analyst,Rolls-Royce,https://in.linkedin.com/jobs/view/financial-an...,"Bengaluru, Karnataka, India",Actively Hiring,40 minutes ago
2,store accelerator programme: business trainee,CRED,https://in.linkedin.com/jobs/view/store-accele...,"Bengaluru, Karnataka, India",Be an early applicant,9 hours ago
3,Sales Manager I,Swiggy,https://in.linkedin.com/jobs/view/sales-manage...,"Bengaluru, Karnataka, India",Actively Hiring,14 hours ago
4,Bioinformatics part time job/internship at Ban...,BayesLabs,https://in.linkedin.com/jobs/view/bioinformati...,"Bengaluru, Karnataka, India",Be an early applicant,17 hours ago
...,...,...,...,...,...,...
995,Analyst,PwC India,https://in.linkedin.com/jobs/view/analyst-at-p...,"Bengaluru, Karnataka, India",Actively Hiring,18 hours ago
996,Customer Support Intern,Truecaller,https://in.linkedin.com/jobs/view/customer-sup...,"Bengaluru, Karnataka, India",Be an early applicant,6 minutes ago
997,Strategic Sourcing and Vendor Management,Yulu,https://in.linkedin.com/jobs/view/strategic-so...,"Bengaluru, Karnataka, India",Actively Hiring,22 hours ago
998,"Software Engineer (Backend, 7-10 years)",PhonePe,https://in.linkedin.com/jobs/view/software-eng...,"Bengaluru, Karnataka, India",Actively Hiring,15 hours ago


In [119]:
df['Posted'] = df['Posted'].apply(convert_post)


# for i in range(len(df['Posted'])):
#     out = convert_post(df['Posted'][i])
#     df['Posted'][i]= out

# df

In [120]:
df

,Title,Company,Company Link,Location,Status,Posted
0,Sales Specialist,Amazon,https://in.linkedin.com/jobs/view/sales-specia...,"Bengaluru, Karnataka, India",Actively Hiring,2022-06-27 18:02:16.794719
1,Financial Analyst,Rolls-Royce,https://in.linkedin.com/jobs/view/financial-an...,"Bengaluru, Karnataka, India",Actively Hiring,2022-06-26 21:02:16.794771
2,store accelerator programme: business trainee,CRED,https://in.linkedin.com/jobs/view/store-accele...,"Bengaluru, Karnataka, India",Be an early applicant,2022-06-28 04:02:16.794775
3,Sales Manager I,Swiggy,https://in.linkedin.com/jobs/view/sales-manage...,"Bengaluru, Karnataka, India",Actively Hiring,2022-06-27 23:02:16.794778
4,Bioinformatics part time job/internship at Ban...,BayesLabs,https://in.linkedin.com/jobs/view/bioinformati...,"Bengaluru, Karnataka, India",Be an early applicant,2022-06-27 20:02:16.794781
...,...,...,...,...,...,...
995,Analyst,PwC India,https://in.linkedin.com/jobs/view/analyst-at-p...,"Bengaluru, Karnataka, India",Actively Hiring,2022-06-27 19:02:16.797546
996,Customer Support Intern,Truecaller,https://in.linkedin.com/jobs/view/customer-sup...,"Bengaluru, Karnataka, India",Be an early applicant,2022-06-28 07:02:16.797549
997,Strategic Sourcing and Vendor Management,Yulu,https://in.linkedin.com/jobs/view/strategic-so...,"Bengaluru, Karnataka, India",Actively Hiring,2022-06-27 15:02:16.797552
998,"Software Engineer (Backend, 7-10 years)",PhonePe,https://in.linkedin.com/jobs/view/software-eng...,"Bengaluru, Karnataka, India",Actively Hiring,2022-06-27 22:02:16.797555
